# Import des librairies

In [ ]:
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
import numpy as np
import importlib

spec = importlib.util.spec_from_file_location("fspliter", "..\\utils\\files_spliter.py")
fspliter = importlib.util.module_from_spec(spec)
spec.loader.exec_module(fspliter)

spec = importlib.util.spec_from_file_location("preprocessing", "..\\utils\\preprocessing.py")
preprocessing = importlib.util.module_from_spec(spec)
spec.loader.exec_module(preprocessing)

# Définition de fonction utilitaire

In [ ]:
def count_outliers(arr):
    return np.count_nonzero(arr == -1)

In [ ]:
def count_inliners(arr):
    return np.count_nonzero(arr == 1)

# Lecture d'un fichier de données

In [ ]:
data = fspliter.get_mice(0)
data

# Détection des outliers pour les valeurs numériques

## Filtre les colonnes numériques

In [ ]:
data = data.drop(['rawState', 'state', 'filename', 'temp'], axis=1)

## Tests de méthode de scikit learn

In [ ]:
methods = ["Isolation Forest", "Local outlier Factor"]
clfs = [IsolationForest(n_estimators=10), LocalOutlierFactor()]

In [ ]:
for i in range(len(methods)):
    clf = clfs[i]
    method = methods[i]
    outliers = clf.fit_predict(data)
    print("Méthode " + method + ":")
    print("- " + str(count_outliers(outliers)) + " ouliers trouvés")
    print("- " + str(count_inliners(outliers)) + " inliners trouvés")
    print()

## Test de la méthode IQR

In [ ]:
print("Nombre d'outliers par colonnes:")
cols = ['EEGv', 'EMGv']
bins = data.drop(cols, axis=1)
mean_outliers_bins = 0
clean_outlier = data

threshold = 2

for column in cols:
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1

    outliers = data[(data[column] < Q1 - threshold * IQR) | (data[column] > Q3 + threshold * IQR)]
    clean_outlier = clean_outlier[(clean_outlier[column] >= Q1 - threshold * IQR) & (clean_outlier[column] <= Q3 + threshold * IQR)]
    print(column + ": " + str(len(outliers)))

for column in bins:
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1

    outliers = data[(data[column] < Q1 - threshold * IQR) | (data[column] > Q3 + threshold * IQR)]
    clean_outlier = clean_outlier[(clean_outlier[column] >= Q1 - threshold * IQR) & (clean_outlier[column] <= Q3 + threshold * IQR)]
    mean_outliers_bins = mean_outliers_bins + len(outliers)/len(bins.columns)

print("Nombre moyen d'outliers pour les colonnes bin: " + str(round(mean_outliers_bins)))
print("==================================")
print("Nombre de ligne enlevée par cette méthode: " + str(len(data) - len(clean_outlier)))

# Détection des outliers pour les valeurs non numériques
Il y a uniquement la colonne `state` qui contient des valeurs non numériques pertinantes. Cette colonne permet de classer l'état de sommeil de souris en trois classe `w`, `r` et `n` toutes autres classes sont considérées comme outlier.


# Conclusion
- La méthode de détection des outliers avec `IQR` n'est pas efficace dans notre cas. Cette méthode enlève 86'000 lignes sur 86'400.
- Les deux méthodes testées de scikit learn ont l'air plus intéressante. Notamment la méthode `Isolation Forest` qui est recommandée pour les times series.